In [ ]:
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install wandb

In [1]:
from datasets import Audio, ClassLabel, load_dataset, load_metric
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split
from torch._C import device
# from torch.utils.data import Dataset
from transformers import (
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    trainer_utils
)

In [1]:
import json
import os
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import IPython.display as ipd
# import jiwer
import librosa
import numpy as np
import pandas as pd
import random
import re
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
import torchaudio
# import wandb


c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print('Start')

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
WANDB_KEY = os.getenv('WANDB_KEY')

In [2]:
wandb.login(key=WANDB_KEY) # add wandb API key
wandb.init(project="speech")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ghfn. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Acer/.netrc


In [4]:
import os

# os.mkdir("/kaggle/working/ne_np_female/")

In [3]:
colnames = ["path", "labels"] 
df = pd.read_csv("./input/ne_np_female/line_index.tsv", sep = "\t", header = None, names = colnames)
df["path"] = "./input/ne_np_female/wavs/" + df["path"] + ".wav"

In [4]:
train_df, test_df = train_test_split(df, random_state = 0, train_size = .80)
train_df, val_df = train_test_split(df, random_state = 0, train_size = .75)

In [5]:
os.makedirs('./ne-np-female')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: './ne-np-female'

In [6]:
train_df.to_csv("./ne-np-female/line_index_train.csv", index = False)
val_df.to_csv("./ne-np-female/line_index_val.csv", index = False)
test_df.to_csv("./ne-np-female/line_index_test.csv", index = False)

In [7]:
data_files = {
    "train": "line_index_train.csv",
    "validation": "line_index_val.csv",
    "test": "line_index_test.csv"
}

In [8]:
train_data = load_dataset("./ne-np-female", data_files = data_files, split = "train")
val_data = load_dataset("./ne-np-female", data_files = data_files, split = "validation")

Using custom data configuration ne-np-female-9337105c2b0df777


Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 1500.47it/s]
0 tables [00:00, ? tables/s]c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
0 tables [00:00, ? tables/s]c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
0 tables [00:00, ? tables/s]c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the '

Dataset csv downloaded and prepared to C:/Users/Acer/.cache/huggingface/datasets/csv/ne-np-female-9337105c2b0df777/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [9]:
def show_random_elements(dataset, num_examples = 10):
    assert num_examples <=  len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(train_data.remove_columns(["path"]), num_examples = 20)

,labels
0,उन्नाइस सय सत्तरीमा कोलम्बियाली फुटबल व्यावसायिकको पहिलो श्रेणी
1,दुई हजार अट्ठाइसमा धानका बाला इलरी बिलरी गीतमा नाच्ने उहाँ पहिलोपटक मञ्चमा पुग्नुभएको हो
2,जर्मनीको मार्किस्कर क्रेइसमा रहेको नगरपालिका
3,खटजात्राक्रममा पुजारीद्वारा घण्टी बजाउने चमर डोलाउने कार्य अविच्छिन्न भइरहेको हुन्छ
4,उपकरण जसले डिजिटल जानकारी इनकोड गर्न एनालग वाहक सङ्केत नियन्त्रण गर्दछ
5,फेडेक्स कर्पोरेसनको प्रमुख कार्यकारिणीको सुरक्षामा ठुलो रकम खर्चिनु पर्‍यो
6,द्विपक्षीय सन्तुलन भएका सबै जनावरहरू
7,भवनको वजन धान्ने संरचनात्मक तत्व
8,रजिना कस्मेटिक एण्ड टेलरिङ्गमा जानुहोस् र मनपर्ने कुराहरू लिनुहोस्
9,कम्प्युटर लागि इलेक्ट्रोनिक दृश्य प्रदर्शन


In [11]:
chars_to_ignore = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["labels"] = re.sub(chars_to_ignore, '', batch["labels"]).lower() + " "
    return batch

In [12]:
train_data = train_data.map(remove_special_characters)
val_data = val_data.map(remove_special_characters)

100%|██████████| 516/516 [00:00<00:00, 39690.82ex/s]


In [13]:
show_random_elements(train_data.remove_columns(["path"]))

,labels
0,हामी सधैँ आदर्श किराना पसलमा सामान किन्ने गर्दछौँ
1,जितु नेपाल नेपालका एक चर्चित हाँस्य कलाकार हुन्
2,स्पेनी स्वायत्त समुदाय
3,मुस्ताङ हिमाली जिल्ला हो
4,हालको बाग्लुङ्गको ग्वाली चौरसम्म दक्षिणमा धुर्कोटसम्म र पूर्वमा मुसिकोट राज्यसम्म फैलिएको थियो
5,यी दुवैले सात फिल्म बनाए
6,धालिसको धेरै मात्रामा उत्‍पादन हुन्छ
7,भारतीय कम्युनिस्ट पार्टी जनतादल एस मुस्लिम लिग केरल काङ्ग्रेस एम केरल काङ्ग्रेस जे आदि
8,सुर्खेत जुम्ला सडक यो गाउँबाटै जान्छ यो गाउँलाई पिपलकोटको मुख्य व्यापारिक स्थानमा लिन सकिन्छ
9,चिली राष्ट्रिय फुटबल टिम र वारेन म्युनिखका लागि मिडफिल्डर फुटबलरको जन्म बाइस मे उन्नाइस सय सतासीमा भएको हो


In [14]:
tokenizer = Wav2Vec2CTCTokenizer(
    "./input/beamlab-vocab/vocab.json", 
    unk_token = "[UNK]", 
    pad_token = "[PAD]", 
    word_delimiter_token = "|"
)

In [15]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size = 1, 
    sampling_rate = 16000, 
    padding_value = 0.0, 
    do_normalize = True, 
    return_attention_mask = True
)

In [16]:
processor = Wav2Vec2Processor(
    feature_extractor = feature_extractor, 
    tokenizer = tokenizer
)

In [17]:
processor.save_pretrained("./processor/")

In [18]:
train_data[0]

{'path': './input/ne_np_female/wavs/nep_5687_6685827002.wav',
 'labels': 'नमोबुद्ध कस्मेटिक एण्ड टेलरिङ्ग सेन्टरबाट सिलाइएका लुगाहरू मिलेका र राम्रा हुन्छन् '}

In [19]:
val_data

Dataset({
    features: ['path', 'labels'],
    num_rows: 516
})

In [20]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["labels"]
    
    resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    
    return batch

In [21]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
val_data = val_data.map(speech_file_to_array_fn, remove_columns=val_data.column_names)

100%|██████████| 516/516 [00:02<00:00, 217.08ex/s]


In [24]:
# def resample(batch):
#     batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16000)
#     batch["sampling_rate"] = 16000
#     return batch

In [25]:
# train_data = train_data.map(resample, num_proc=4)
# val_data = val_data.map(resample, num_proc=4)

In [22]:
rand_int = random.randint(0, len(train_data) - 1)
ipd.Audio(data = np.asarray(
    train_data[rand_int]["speech"]), 
    autoplay = True, 
    rate = 16000
)

In [23]:
rand_int = random.randint(0, len(train_data) - 1)

print("Target text:", train_data[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_data[rand_int]["speech"]).shape)
print("Sampling rate:", train_data[rand_int]["sampling_rate"])

Target text: अहिले बलराम चेन्नाइको पैसा लगानी गर्ने तालमेल मिलाउँदैछ 
Input array shape: (78384,)
Sampling rate: 16000


In [24]:
def prepare_dataset(batch, processor):
        
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate = batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [25]:
train_data = train_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = train_data.column_names, batch_size = 8, num_proc = 4, batched = True)
val_data = val_data.map(prepare_dataset, fn_kwargs = {"processor": processor}, remove_columns = val_data.column_names, batch_size = 8, num_proc = 4, batched = True)

In [26]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [27]:
data_collator = DataCollatorCTCWithPadding(processor = processor, padding = True)

In [28]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer", revision = "master")

C:\Users\Acer\AppData\Local\Temp\ipykernel_19824\3898381163.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [29]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    
    # cer_best = 1 - cer because load best model is considering greater value of cer for better results
    cer_best = 1 - cer
    # wandb.log({'wer':wer, 'cer':cer})
    # experiment.log_metric('wer', wer)
    # experiment.log_metric('cer', cer)
    
    return {"wer": wer, "cer": cer, "cer_best": cer_best}

In [30]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout = 0.1,
    hidden_dropout = 0.1,
    feat_proj_dropout = 0.0,
    mask_time_prob = 0.05,
    layerdrop = 0.1,
    gradient_checkpointing = True, 
    ctc_loss_reduction = "mean", 
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.codevectors', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [31]:
model.freeze_feature_extractor()

c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1635: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [1]:
training_args = TrainingArguments(
  output_dir = "./model",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16 = True
  save_steps = 144,
  eval_steps=144,
  logging_steps=144,
  learning_rate=3e-4,
  warmup_steps=180,
  save_total_limit = 2,
  overwrite_output_dir=True,
  gradient_checkpointing = True,
  report_to = "wandb",
  metric_for_best_model = 'cer_best',
  load_best_model_at_end=True
)

SyntaxError: invalid syntax (254856494.py, line 9)

In [37]:
import tensorflow as tf
tf.test.is_built_with_cuda()

True

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:35_Pacific_Daylight_Time_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.relgpu_drvr445TC445_37.28845127_0


In [3]:
!nvidia-smi

Tue Oct 11 12:24:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.48       Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8     6W /  N/A |    355MiB /  4096MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = processor.feature_extractor,
)

In [35]:
train_data

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 1547
})

In [43]:
trainer.train()

c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1547
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1440
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
  0%|          | 0/1440 [00:00<?, ?it/s]c:\Users\Acer\anaconda3\envs\kaggle\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:154: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` me

In [40]:
trainer.save_model("./model")

Saving model checkpoint to ./model
Configuration saved in ./model/config.json
Model weights saved in ./model/pytorch_model.bin
Feature extractor saved in ./model/preprocessor_config.json


In [41]:
model = Wav2Vec2ForCTC.from_pretrained("./model").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("./processor")

loading configuration file ./model/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-large-xlsr-53",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_

In [42]:
input_dict = processor(val_data["input_values"][0], return_tensors = "pt", padding = True)

logits = model(input_dict.input_values.to("cuda")).logits

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [43]:
pred_ids = torch.argmax(logits, dim = -1)[0]

In [44]:
test_transcription_data = load_dataset("./ne-np-female", data_files = data_files, split = "test")

In [45]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_transcription_data["labels"][0].lower())

Prediction:
कालोटाउके गंगा चिल नेपालमा पाइने एक प्रकारको चराको नाम हो

Reference:
कालोटाउके गंगाचील नेपालमा पाइने एक प्रकारको चराको नाम हो


In [46]:
input_dict = processor(val_data["input_values"][1], return_tensors = "pt", padding = True)

logits = model(input_dict.input_values.to("cuda")).logits

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [47]:
pred_ids = torch.argmax(logits, dim = -1)[0]

In [48]:
print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(test_transcription_data["labels"][1].lower())

Prediction:
ताइसन फुरट्स इन्कले विभिन्न प्रकारका खाद्यानहरू उपलब्त गरौँदैई आएको छ

Reference:
टाइसन फुड्स इन्कले विभिन्न प्रकारका खाद्यान्नहरू उपलब्ध गराउँदै आएको छ
